In [ ]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 4),sharex=True, facecolor="none")

for name, tag in [
    ('pils-v2-*-adam-v2', 'SparseDS-Adam'),
    ('pils-v2-*-ekf-v3', 'SparseDS-EKF'),
    # ('pils-v2-*-ekf-v4', 'SparseDS-EKF-1-1000'),
    # ('pils-v2-*-ekf-v5', 'SparseDS-EKF-1-10000'),
    ('pils-v5-*-adam-v2-pre', 'DenseDS-Adam'),
    ('pils-v5-*-ekf-v3-pre', 'DenseDS-EKF'),
]:
    logs = glob(f'../models/{name}-*/pinn.log')
    color = next(ax1._get_lines.prop_cycler)['color']
    for log in logs:
        step, e_err, _, f_err = np.loadtxt(log).T
        ax1.plot(step, e_err*1e3, label=tag, color=color)
        ax2.plot(step, f_err*1e3, label=tag, color=color)
        
        
ax1.set_ylabel('E RMSE [meV/atom]')
ax2.set_ylabel('F RMSE [meV/$\AA$]')
ax1.grid(':')
ax2.grid(':')
ax1.set_yscale('log')
ax1.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xscale('log')
ax1.set_ylim([0.5,50])
ax2.set_ylim([20,500])

unique_legend(ax1, bbox_to_anchor=[1,1], loc=2)
plt.savefig('training-log.png', facecolor='w')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

def unique_legend(ax, **kwargs):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique), **kwargs)